# 读取数据
今天要做的事情：<br>
&emsp;&emsp;    1 先直接用LR,GBDT 出一版baseline结果。<br>
&emsp;&emsp;    2 分别采用bagging, boosting, stacking 出结果<br>
    
下一步<br>
&emsp;&emsp;    1 研究每个字段数值特征：是否需要归一化，中心化，标准化 <br>
&emsp;&emsp;    2 尝试特征筛选<br>
&emsp;&emsp;    3 特征工程PCA、LDA、SVD 降维<br>
&emsp;&emsp;    4 尝试特征升维？？神经网络？？<br>
&emsp;&emsp;    5 加入深度学习方法<br>
&emsp;&emsp;    6 考虑时序怎么用、、<br>

In [1]:
import pandas as pd
train_df = pd.read_csv('./train.csv', index_col='ID')
train_df.head()

,V_Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V22,V23,V24,V25,V26,V27,V28,V29,V30,Label
ID,,,,,,,,,,,,,,,,,,,,,
254359,156699,-0.935008,0.820946,1.067777,0.288612,1.350836,-0.104075,1.021084,-0.009889,-1.356626,...,0.915678,-0.501440,0.754770,1.084112,-0.259672,0.026528,0.051355,0.042881,0.771583,0
244959,152554,2.039188,-0.264982,-1.235053,0.239175,-0.125661,-0.756202,-0.068806,-0.041701,0.659533,...,-0.830929,0.313942,-0.392227,-0.352006,0.209928,-0.079692,-0.055856,0.002934,-1.225918,0
79483,58048,-0.377984,0.917614,1.714673,0.583275,0.887798,0.045338,1.216416,-0.684746,0.012233,...,-0.123676,-0.076580,-0.305154,-0.614375,-0.675006,-0.320966,-0.407508,0.013761,-3.124638,0
164477,116748,1.985660,-0.752667,-1.669258,-0.634638,1.679647,3.937192,-1.309168,1.055526,1.108597,...,0.561408,0.196288,0.755987,-0.226402,0.589655,-0.016130,-0.049041,0.006732,-2.550306,0
184542,126292,1.930330,2.563490,-4.759537,4.675617,2.921796,-1.740102,1.323942,-0.361470,-2.267007,...,-0.698716,-0.192089,-1.056597,0.444520,0.144779,0.013315,0.109086,0.000068,0.139250,0


In [3]:
train_feature = train_df[['V1','V2','V3','V4','V5','V6','V7','V8','V9','V10','V11','V12','V13','V14','V15','V16','V17','V18','V19','V20','V21','V22','V23','V24','V25','V26','V27','V28','V29','V30']]
train_label = train_df['Label']

print train_feature.head()
print train_label.head()
print train_label.value_counts()

              V1        V2        V3        V4        V5        V6        V7  \
ID                                                                             
254359 -0.935008  0.820946  1.067777  0.288612  1.350836 -0.104075  1.021084   
244959  2.039188 -0.264982 -1.235053  0.239175 -0.125661 -0.756202 -0.068806   
79483  -0.377984  0.917614  1.714673  0.583275  0.887798  0.045338  1.216416   
164477  1.985660 -0.752667 -1.669258 -0.634638  1.679647  3.937192 -1.309168   
184542  1.930330  2.563490 -4.759537  4.675617  2.921796 -1.740102  1.323942   

              V8        V9       V10    ...          V21       V22       V23  \
ID                                      ...                                    
254359 -0.009889 -1.356626 -0.384819    ...     0.365994  0.915678 -0.501440   
244959 -0.041701  0.659533  0.309476    ...    -0.295161 -0.830929  0.313942   
79483  -0.684746  0.012233  0.746790    ...    -0.255318 -0.123676 -0.076580   
164477  1.055526  1.108597  0.032542   

In [4]:
pred_df = pd.read_csv('./pred.csv', index_col='ID')
del pred_df['V_Time']
pred_df.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30
ID,,,,,,,,,,,,,,,,,,,,,
54043,-2.389003,0.508246,0.955227,0.355585,-1.346217,0.052071,-1.140844,1.563165,-0.141766,-1.020786,...,0.190740,0.019433,-0.096179,0.025572,-0.271135,0.249452,-0.487984,-0.221525,0.035281,-2.741180
100632,1.324636,0.095398,-0.105591,0.246712,0.320204,0.257465,-0.117373,0.073556,0.265089,-0.096078,...,-0.335731,-0.955960,-0.004008,-1.334493,0.290240,0.219790,-0.000750,-0.015829,0.002406,-7.049453
260528,-0.083895,0.543350,-0.244593,-0.750945,0.804470,-0.429098,1.249977,-0.179319,-0.058560,-0.495834,...,-0.264119,-0.738258,0.164438,-1.005575,-0.293204,0.179689,0.218246,0.086205,0.112603,-1.469655
244435,2.027800,-0.080261,-1.167567,0.210231,0.136417,-0.676231,0.137029,-0.212528,0.176972,0.185304,...,-0.255778,-0.572261,0.282490,-0.319414,-0.266087,0.195817,-0.061286,-0.068097,0.011066,-0.322347
25470,-0.621263,-0.400795,1.599899,-0.959784,0.457530,0.033135,-0.015273,-0.128394,-1.793275,0.764628,...,-0.440350,-0.884314,-0.157001,-0.797327,0.016989,1.121202,-0.180910,-0.167883,0.049452,-2.212872


# LR

## 模型

In [5]:
from sklearn import linear_model
import time

start = time.time()
# fit到RandomForestRegressor之中
clf = linear_model.LogisticRegression(C=1.0, penalty='l1', tol=1e-6)
clf.fit(train_feature, train_label)

print "运行时间：", time.time()-start

运行时间： 3.40825891495


## 评估 

In [7]:
from sklearn.metrics import accuracy_score
lr_pred_label = pd.Series(clf.predict(pred_df))
lr_pred_label.value_counts()

0    99845
1      155
dtype: int64

#  RF

In [8]:
from sklearn.ensemble import RandomForestClassifier
import time

start = time.time()
aliRC_RFC = RandomForestClassifier()
aliRC_RFC = aliRC_RFC.fit(train_feature, train_label)
print "训练时间：", time.time()-start

训练时间： 7.86861419678


In [9]:
rf_pred_label = pd.Series(aliRC_RFC.predict(pred_df))
rf_pred_label.value_counts()

0    99837
1      163
dtype: int64

# GBDT

In [10]:
from sklearn.ensemble import GradientBoostingClassifier
import time

start = time.time()
aliRC_gdbt = GradientBoostingClassifier()
aliRC_gdbt = aliRC_gdbt.fit(train_feature, train_label)
print "训练时间：", time.time()-start

训练时间： 66.1180150509


In [12]:
gbdt_pred_label = pd.Series(aliRC_gdbt.predict(pred_df))
gbdt_pred_label.value_counts()

0    99932
1       68
dtype: int64

# GXBOOST

In [13]:
from xgboost import XGBClassifier
import time

start = time.time()
xgbc = XGBClassifier()
xgbc.fit(train_feature, train_label)
print "训练时间：", time.time()-start

训练时间： 28.6355919838


In [17]:
xgbc_pred_lable = pd.Series(xgbc.predict(pred_df))
gbdt_pred_label.value_counts()

0    99932
1       68
dtype: int64